In [ ]:
import re
import nltk
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import json


In [ ]:
# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [1]:
from google.colab import files

uploaded = files.upload()

Saving dataset.json to dataset.json


In [3]:
import json

with open('dataset.json', 'r') as file:
    data = json.load(file)
print(data)

[{'Incorrect': 'මම සෙල්ලම් කරයි', 'Correct': 'මම සෙල්ලම් කරමි'}, {'Incorrect': 'මම පාසල් යමු', 'Correct': 'මම පාසල් යමි'}, {'Incorrect': 'අපි ගෙදර වැඩ කරමි', 'Correct': 'අපි ගෙදර වැඩ කරමු'}, {'Incorrect': 'ඔහු ගමට යති', 'Correct': 'ඔහු ගමට යයි'}, {'Incorrect': 'ඇය ඇඳුම් මසති', 'Correct': 'ඇය ඇඳුම් මසයි'}, {'Incorrect': 'අපි නිවසට ගමන් කරයි', 'Correct': 'අපි නිවසට ගමන් කරමු'}, {'Incorrect': 'මම පාඩම් කරති', 'Correct': 'මම පාඩම් කරමි'}, {'Incorrect': 'අපි ඉගෙන ගමි', 'Correct': 'අපි ඉගෙන ගමු'}, {'Incorrect': 'ඔහු අපට උගන්නති', 'Correct': 'ඔහු අපට උගන්නයි'}, {'Incorrect': 'මම ඔහුගෙන් ඉගෙන ගමු', 'Correct': 'මම ඔහුගෙන් ඉගෙන ගමි'}, {'Incorrect': 'ඇය නිවසට යමු', 'Correct': 'ඇය නිවසට යයි'}, {'Incorrect': 'ඔහු වාහනය එලවති', 'Correct': 'ඔහු වාහනය එලවයි'}, {'Incorrect': 'අපි චිත්\u200dර අඳිමි', 'Correct': 'අපි චිත්\u200dර අඳිමු'}, {'Incorrect': 'මම සින්දු කියති', 'Correct': 'මම සින්දු කියමි'}, {'Incorrect': 'මම ඔහුව තල්ලු කරමු', 'Correct': 'මම ඔහුව තල්ලු කරමි'}, {'Incorrect': 'අපි මුහුදේ පීනති', '

In [ ]:
# Step 1: Preprocess Sentences
def preprocess_sentence(sentence):
    """Clean, tokenize, and split the sentence."""
    sentence = re.sub(r"[^\w\s]", "", sentence)  # Remove unwanted characters
    tokens = nltk.word_tokenize(sentence)       # Tokenize into words
    return tokens

In [ ]:
# Step 2: Feature Extraction
def extract_features(sentence):
    """Extract features for the machine learning model."""
    tokens = preprocess_sentence(sentence)
    features = " ".join(tokens)  # Convert tokens back to a string for vectorization
    return features

In [ ]:
# Prepare dataset
X = [extract_features(item["sentence"]) for item in data]
y = [1 if item["is_correct"] else 0 for item in data]

In [ ]:
# Vectorize text data
vectorizer = CountVectorizer()
X_vect = vectorizer.fit_transform(X)

In [ ]:
# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_vect, y, test_size=0.2, random_state=42)

# Step 3: Train Decision Tree Classifier
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

DecisionTreeClassifier()

In [ ]:
# Evaluate the Model
y_pred = clf.predict(X_test)
print("Model Accuracy:", accuracy_score(y_test, y_pred))

Model Accuracy: 0.3333333333333333


In [ ]:
y_pred = clf.predict(X_train)
print("Model Accuracy:", accuracy_score(y_train, y_pred))

Model Accuracy: 1.0


In [ ]:
# Step 4: Grammar Error Detection and Correction
def suggest_corrections(sentence, dataset):
    """Provide grammatical suggestions for incorrect sentences."""
    # Check if the sentence is grammatically correct
    if clf.predict(vectorizer.transform([extract_features(sentence)]))[0] == 1:
        print("Sentence is Grammatically Correct!")
        return {"present": sentence, "future": sentence, "past": sentence} # Return a dictionary when the sentence is correct
    else:
        print("Grammar Error Detected!")
        # Find the sentence in the dataset
        for item in dataset:
            if item["sentence"] == sentence:
                suggestions = item["suggestions"]
                return suggestions
        return {"error": "Suggestions not available for this sentence."}

In [ ]:
# Step 5: Test the Correction System
test_sentence = "කමල් සමග මම පාඩම් කරමි"
corrections = suggest_corrections(test_sentence, data)

print("Original Sentence:", test_sentence)
print("Corrections:")
print("Present Tense:", corrections.get("present", "No suggestion"))
print("Future Tense:", corrections.get("future", "No suggestion"))
print("Past Tense:", corrections.get("past", "No suggestion"))

Sentence is Grammatically Correct!
Original Sentence: කමල් සමග මම පාඩම් කරමි
Corrections:
Present Tense: කමල් සමග මම පාඩම් කරමි
Future Tense: කමල් සමග මම පාඩම් කරමි
Past Tense: කමල් සමග මම පාඩම් කරමි


In [ ]:
# Step 5: Test the Correction System
test_sentence = ""
corrections = suggest_corrections(test_sentence, data)

print("Original Sentence:", test_sentence)
print("Corrections:")
print("Present Tense:", corrections.get("present", "No suggestion"))
print("Future Tense:", corrections.get("future", "No suggestion"))
print("Past Tense:", corrections.get("past", "No suggestion"))

Sentence is Grammatically Correct!
Original Sentence: ඇය විසිහතර පැයේ ම කාලසටහනකට අනුව වැඩ කරමු
Corrections:
Present Tense: ඇය විසිහතර පැයේ ම කාලසටහනකට අනුව වැඩ කරමු
Future Tense: ඇය විසිහතර පැයේ ම කාලසටහනකට අනුව වැඩ කරමු
Past Tense: ඇය විසිහතර පැයේ ම කාලසටහනකට අනුව වැඩ කරමු


In [ ]:
test_sentence = "ප්‍රාථමික අධ්‍යාපනය අවසන් කළ කුඩා එඩ්මන්ඩ් පීරිස් කොළඹ සාන්ත ජෝසප් විද්‍යාලයට ඇතුළත්ව එහි දීප්තිමත් ශිෂ්‍යයකු වශයෙන් උසස් ලෙස සමත්කම් දක්වමින් ද්විතීයක අධ්‍යාපනය නිම කළේමු"
corrections = suggest_corrections(test_sentence, data)

print("Original Sentence:", test_sentence)
print("Corrections:")
print("Present Tense:", corrections.get("present", "No suggestion"))
print("Future Tense:", corrections.get("future", "No suggestion"))
print("Past Tense:", corrections.get("past", "No suggestion"))

Grammar Error Detected!
Original Sentence: ප්‍රාථමික අධ්‍යාපනය අවසන් කළ කුඩා එඩ්මන්ඩ් පීරිස් කොළඹ සාන්ත ජෝසප් විද්‍යාලයට ඇතුළත්ව එහි දීප්තිමත් ශිෂ්‍යයකු වශයෙන් උසස් ලෙස සමත්කම් දක්වමින් ද්විතීයක අධ්‍යාපනය නිම කළේමු
Corrections:
Present Tense: ප්‍රාථමික අධ්‍යාපනය අවසන් කළ කුඩා එඩ්මන්ඩ් පීරිස් කොළඹ සාන්ත ජෝසප් විද්‍යාලයට ඇතුළත්ව එහි දීප්තිමත් ශිෂ්‍යයකු වශයෙන් උසස් ලෙස සමත්කම් දක්වමින් ද්විතීයක අධ්‍යාපනය නිම කරයි
Future Tense: ප්‍රාථමික අධ්‍යාපනය අවසන් කළ කුඩා එඩ්මන්ඩ් පීරිස් කොළඹ සාන්ත ජෝසප් විද්‍යාලයට ඇතුළත්ව එහි දීප්තිමත් ශිෂ්‍යයකු වශයෙන් උසස් ලෙස සමත්කම් දක්වමින් ද්විතීයක අධ්‍යාපනය නිම කරන්නේ ය
Past Tense: ප්‍රාථමික අධ්‍යාපනය අවසන් කළ කුඩා එඩ්මන්ඩ් පීරිස් කොළඹ සාන්ත ජෝසප් විද්‍යාලයට ඇතුළත්ව එහි දීප්තිමත් ශිෂ්‍යයකු වශයෙන් උසස් ලෙස සමත්කම් දක්වමින් ද්විතීයක අධ්‍යාපනය නිම කළේ ය. 


In [ ]:
test_sentence ="ඔහු පසු කලෙක සිය නිර්මාණ ජිවිතයට එම අත් දැකීම් පිටිවහලක් කර ගත්තෝය"
corrections = suggest_corrections(test_sentence, data)

print("Original Sentence:", test_sentence)
print("Corrections:")
print("Present Tense:", corrections.get("present", "No suggestion"))
print("Future Tense:", corrections.get("future", "No suggestion"))
print("Past Tense:", corrections.get("past", "No suggestion"))

Grammar Error Detected!
Original Sentence: ඔහු පසු කලෙක සිය නිර්මාණ ජිවිතයට එම අත් දැකීම් පිටිවහලක් කර ගත්තෝය
Corrections:
Present Tense: No suggestion
Future Tense: No suggestion
Past Tense: No suggestion


In [ ]:
test_sentence = "ඔවුහු කතාබහ කිරීම ප්‍රතික්ෂේප කරති"
corrections = suggest_corrections(test_sentence, data)

print("Original Sentence:", test_sentence)
print("Corrections:")
print("Present Tense:", corrections.get("present", "No suggestion"))
print("Future Tense:", corrections.get("future", "No suggestion"))
print("Past Tense:", corrections.get("past", "No suggestion"))

Sentence is Grammatically Correct!
Original Sentence: ඔවුහු කතාබහ කිරීම ප්‍රතික්ෂේප කරති
Corrections:
Present Tense: ඔවුහු කතාබහ කිරීම ප්‍රතික්ෂේප කරති
Future Tense: ඔවුහු කතාබහ කිරීම ප්‍රතික්ෂේප කරති
Past Tense: ඔවුහු කතාබහ කිරීම ප්‍රතික්ෂේප කරති


In [ ]:
test_sentence = "ශිෂ්‍යයෝ ඔහුගෙන් ඉගෙන ගමු"
corrections = suggest_corrections(test_sentence, data)

print("Original Sentence:", test_sentence)
print("Corrections:")
print("Present Tense:", corrections.get("present", "No suggestion"))
print("Future Tense:", corrections.get("future", "No suggestion"))
print("Past Tense:", corrections.get("past", "No suggestion"))

Grammar Error Detected!
Original Sentence: ශිෂ්‍යයෝ ඔහුගෙන් ඉගෙන ගමු
Corrections:
Present Tense: ශිෂ්‍යයෝ ඔහුගෙන් ඉගෙන ගනිති
Future Tense: ශිෂ්‍යයෝ ඔහුගෙන් ඉගෙන ගන්නෝය්
Past Tense: ශිෂ්‍යයෝ ඔහුගෙන් ඉගෙන ගත්තෝය


In [ ]:
test_sentence = "සංගීත විශාරදයෝ ගීත ගයති"
corrections = suggest_corrections(test_sentence, data)

print("Original Sentence:", test_sentence)
print("Corrections:")
print("Present Tense:", corrections.get("present", "No suggestion"))
print("Future Tense:", corrections.get("future", "No suggestion"))
print("Past Tense:", corrections.get("past", "No suggestion"))

Sentence is Grammatically Correct!
Original Sentence: සංගීත විශාරදයෝ ගීත ගයති
Corrections:
Present Tense: සංගීත විශාරදයෝ ගීත ගයති
Future Tense: සංගීත විශාරදයෝ ගීත ගයති
Past Tense: සංගීත විශාරදයෝ ගීත ගයති
